In [1]:
import argparse
from helpers import *

In [2]:
## Defining "argparse" parameters
mimic_iv_path = '/Users/jin/Desktop/Temporal_AutoScore/Data/mimic-iv-1.0'
output_path = '/Users/jin/Desktop/Temporal_AutoScore/Data/mimic-iv-1.0'

In [3]:
## Defining auxillary parameters
## Assume that "ed" directory placed under "mimic_iv_path"
mimic_iv_core_path = os.path.join(mimic_iv_path, 'core')
mimic_iv_hosp_path = os.path.join(mimic_iv_path , 'hosp')   
mimic_iv_icu_path = os.path.join(mimic_iv_path, 'icu')
mimic_iv_ed_path = os.path.join(mimic_iv_path, 'ed')

icu_filename_dict = {"chartevents":"chartevents.csv","datetimeevents":"datetimeevents.csv","d_items":"d_items.csv","icustays":"icustays.csv","inputevents":"inputevents.csv","outputevents":"outputevents.csv","procedureevents":"procedureevents.csv"}
core_filename_dict = {"patients":"patients.csv", "admissions":"admissions.csv", "transfers":"transfers.csv"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv","d_icd_diagnoses":"d_icd_diagnoses.csv","d_labitems":"d_labitems.csv","emar":"emar.csv","hcpcsevents":"hcpcsevents.csv","microbiologyevents":"microbiologyevents.csv","poe":"poe.csv","prescriptions":"prescriptions.csv","services":"services.csv","diagnoses_icd":"diagnoses_icd.csv","d_icd_procedures":"d_icd_procedures.csv","drgcodes":"drgcodes.csv","emar_detail":"emar_detail.csv","labevents":"labevents.csv","pharmacy":"pharmacy.csv","poe_detail":"poe_detail.csv","procedures_icd":"procedures_icd.csv"}
ed_filename_dict = {'diagnosis':'diagnosis.csv', 'edstays':'edstays.csv',  'medrecon':'medrecon.csv',  'pyxis':'pyxis.csv',  'triage':'triage.csv',  'vitalsign':'vitalsign.csv'}


In [4]:
## Reading main tables
df_edstays = read_edstays_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['edstays']))
df_patients = read_patients_table(os.path.join(mimic_iv_core_path, core_filename_dict['patients']))
df_admissions = read_admissions_table(os.path.join(mimic_iv_core_path, core_filename_dict["admissions"]))
df_icustays = read_icustays_table(os.path.join(mimic_iv_icu_path, icu_filename_dict['icustays']))
df_triage = read_triage_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['triage']))


In [5]:
## Merging edstays and patients -> merging admissions 
df_edstays = merge_edstays_patients_on_subject(df_edstays ,df_patients)
df_edstays  = merge_edstays_admissions_on_subject(df_edstays ,df_admissions)

In [6]:
## Deriving age and in-patient mortality information
df_edstays = add_age(df_edstays)
df_edstays = add_inhospital_mortality(df_edstays)

In [7]:
## Generate past ED visits
df_edstays = generate_past_ed_visits(df_edstays)

In [8]:
## Generate past admissions
df_edstays = generate_past_admissions(df_edstays, df_admissions)

In [9]:
## Generate past icu visits
df_edstays = generate_past_icu_visits(df_edstays, df_icustays)

In [10]:
## Merging triage vital sign table
df_edstays = merge_edstays_triage_on_subject(df_edstays, df_triage)

print(df_edstays.columns)
print(df_edstays.head())

Index(['index', 'subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime',
       'gender', 'dod', 'admittime', 'dischtime', 'deathtime', 'ethnicity',
       'edregtime', 'edouttime', 'insurance', 'age', 'inhospital_mortality',
       'n_ed', 'n_adm', 'n_icu', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint'],
      dtype='object')
    index  subject_id     hadm_id   stay_id              intime  \
0  335314    10000032  22595853.0  33258284 2180-05-06 19:17:00   
1  335315    10000032  22841357.0  38112554 2180-06-26 15:54:00   
2  335316    10000032  29079034.0  32952584 2180-07-22 16:24:00   
3  335317    10000032  29079034.0  39399961 2180-07-23 05:54:00   
4  335318    10000032  25742920.0  35968195 2180-08-05 20:58:00   

              outtime gender dod           admittime           dischtime  ...  \
0 2180-05-06 23:30:00      F NaT 2180-05-06 22:23:00 2180-05-07 17:15:00  ...   
1 2180-06-26 21:31:00      F NaT 2180-06-26 18: